In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
from dotenv import load_dotenv
from ephys_queries import db_setup_core, select_spike_times, select_discrete_data
from spiketimes.df.binning import spike_count_around_event_by

In [5]:
data_dir = Path(".").absolute().parent / "data"
group_names = ("chronic_citalopram", 
                           "chronic_saline", 
                           "chronic_saline_", 
                           "citalopram_continuation", 
                           "citalopram_discontinuation")
dfb = (
    pd.read_csv(data_dir / "baseline.csv")
    .assign(group= 
            lambda x: x["group_name"].map({"chronic_saline": "CS",
                                          "chronic_saline_": "CS",
                                          "citalopram_continuation": "CC",
                                          "chronic_citalopram": "CC",
                                          "citalopram_discontinuation": "CD"})
           )
    .loc[lambda x: x.group_name.isin(group_names)]
)

In [33]:
block_name = "base_shock"
fs = 30000
load_dotenv()
engine, metadata = db_setup_core()

df_spikes = (
    select_spike_times(
            engine, 
            metadata,
            group_names=group_names,
            block_name=block_name,
            )
    .assign(spiketimes=lambda x: x["spike_time_samples"].divide(fs))
    .merge(dfb[["neuron_id", "session_name"]])
)

df_events = (
    select_discrete_data(
            engine,
            metadata,
            group_names=group_names,
            block_name=block_name
    )
    .assign(event_s= lambda x: x["timepoint_sample"].divide(fs))
)

In [34]:
df_spikes = df_spikes.groupby("neuron_id").filter(lambda x: len(x) >= 500)

In [35]:
df_counts =spike_count_around_event_by(
    df_data=df_spikes,
    binsize=0.35,
    df_data_data_colname="spiketimes",
    df_data_group_colname="session_name",
    df_data_spiketrain_colname="neuron_id",
    df_events=df_events,
    df_events_event_colname="event_s",
    df_events_group_colname="session_name"
)

In [36]:
from itertools import combinations
from scipy.stats import pearsonr
from spiketimes.utils import p_adjust

def pearson_r(
    df, 
    spiketrain_col="spiketrain", 
    data_col="counts", 
    group_col="session_name",
    adjust_p=True,
    p_adjust_method="Benjamini-Hochberg"
):
    """
    Calculate pearson's correlation coefficent between all pairs of
    simultaneously-recorded spiketrains.
    """
    frames = []
    sessions= df[group_col].unique()
    for session in sessions:
        df1 = (
            df
            .loc[df_counts[group_col] == session]
        )
        st_ids = df1[spiketrain_col].unique()
        combs = list(combinations(st_ids, r=2))
        res = list(map(
            lambda x: (x[0], x[1], *pearsonr(df1[df1[spiketrain_col] == x[0]][data_col].values, 
                                        df1[df1[spiketrain_col] == x[1]][data_col].values)),
            combs
        ))
        frames.append(
            pd.DataFrame(res, 
                         columns=["spiketrain_1", "spiketrain_2", "r", "p"]
                        ).assign(**{group_col:session})
        )
    df_res = pd.concat(frames)
    if adjust_p:
        df_res.p = p_adjust(df_res.p)
    return df_res

In [37]:
df_res = pearson_r(df_counts, spiketrain_col="neuron_id", data_col="counts",
         group_col="session_name", adjust_p=True)

In [38]:
df_done = (
    df_res
    .merge(dfb[["neuron_id", "cluster", "group"]], left_on="spiketrain_1", right_on="neuron_id")
    .drop("neuron_id", axis=1)
    .rename(columns={"cluster": "spiketrain_1_cluster"})
    .merge(dfb[["neuron_id", "cluster"]], left_on="spiketrain_2", right_on="neuron_id")
    .drop("neuron_id", axis=1)
    .rename(columns={"cluster": "spiketrain_2_cluster"})
    .assign(has_sr=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "slow_regular") or (y.spiketrain_2_cluster== "slow_regular"),
                       axis=1),
            has_sir=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "slow_irregular") or (y.spiketrain_2_cluster== "slow_irregular"),
                       axis=1),
            has_ff=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "fast_firing") or (y.spiketrain_2_cluster== "fast_firing"),
                       axis=1)
           )
    .assign(comb= lambda x: x.apply(lambda y: 
                                    "sr_sr" if y.has_sr and (not y.has_sir) and (not y.has_ff)
                                   else "sr_sir" if y.has_sr and y.has_sir and (not y.has_ff)
                                   else "sr_ff" if y.has_sr and (not y.has_sir) and y.has_ff
                                   else "sir_sir" if (not y.has_sr) and y.has_sir and (not y.has_ff)
                                   else "sir_ff" if (not y.has_sr) and y.has_sir and y.has_ff
                                   else "ff_ff", axis=1
                                   ))
)

In [52]:
df_done.to_csv(data_dir / "evoked_rsc.csv", index=False)

-0.007350441006729332